In [11]:
def ReduceCluster(S, eps=10**-6, c=1, precision=2000):
    n = S[0].parent().codomain().dimension_relative() + 1
    cluster0 = [vector(list(pnt)) for pnt in S]
    m = len(cluster0)
    C = ComplexField(prec=precision)
    R = RealField(prec=precision)
    for v in cluster0:
        v.change_ring(C)
    cluster = [v*1/R(v.norm()) for v in cluster0]
    Tr = matrix.identity(n)
    count = 0
    total_count = 0
    while total_count < S[1][1].prec()*2:
        total_count += 1
        cols = matrix(cluster).transpose()
        der1 = [col.hermitian_inner_product(col) for col in cols]
        #der1 = [R(i.real()) + R(i.imag()) for i in der1] # fix small errors ???
        #der1 = [i.real().sign()*i.abs() for i in der1]
        der2 = []
        for i in range(n):
            temp_list = []
            for j in range(i):
                inner_prod = cols[i].hermitian_inner_product(cols[j])
                #inner_prod = R(inner_prod.real()) + R(inner_prod.imag())
                #inner_prod = inner_prod.real().sign()*inner_prod.abs()
                temp_list.append(2*(inner_prod))
            der2.append(temp_list)
        average_der1 = sum(der1)/n
        der1 = [r - average_der1 for r in der1]
        absd = 0
        der2_prime = [] # unpack the list der2
        for lst in der2:
            der2_prime += lst
        for d in der1 + der2_prime:
            absd += d**2
        print('|der| =', absd)
        if absd < eps**2:
            break
        for i in range(len(der2)):
            der2[i].append(der1[i])
        
        #print('der2:', der2)
        der3 = []
        for lst in der2:
            new_lst = lst + [C(0)]*(n - len(lst))
            der3.append(new_lst)
            
        # make symmetric matrix
        B1 = matrix(der3)
        for i in range(n-1):
            for j in range(1 + i, n):
                B1[i, j] = B1[j, i]
        #print('B1:', B1)
        B = MatExp(-c*B1, eps=eps*1**-10).change_ring(R)
        #print('B:', B)
        B *= B.determinant()**(-1/n)
        
        Tr *= B
        cluster1 = [B*pnt for pnt in cluster]
        val = sum(pnt.hermitian_inner_product(pnt).log() for pnt in cluster1)
        print('gain =', val)
        if val > 0:
            c /= 2
            print('new c =', c)
        else:
            count += 1
            if count == 10:
                count = 0
                Tr *= R(Tr.det())**(-1/n)
                cluster1 = [pt*Tr for pt in cluster0]
            cluster = [v*1/R(v.norm()) for v in cluster1]
    Tr = Tr*Tr.transpose()
    Tr = Tr.change_ring(RealField(prec=100))
    Tr = (Tr + Tr.transpose())/2
    U1 = Tr.LLL_gram().transpose()
    U1_inv = U1.inverse()
    return [v*U1_inv for v in cluster0], U1_inv, U1

In [9]:
def MatExp(mat, eps=1*10**-6):
    
    def mat_prod(mat):
        lst = []
        for tup in mat:
            for ele in tup:
                lst.append(ele)
        vec = vector(lst)
        return vec.inner_product(vec)
        
    n = len(mat[0])
    result = matrix.identity(n)
    k = 1
    m = result
    while mat_prod(mat) > eps^2:
        m *= 1/k * mat
        result += m
        k += 1
        #rint('m:', m)
        #rint('k:', k)
    return result

In [27]:
x = (C(1)/C(-8+i))
x

-0.16666666666666666666666666667

In [15]:
m = matrix(RR, 3, 3, [367751.9942, -254909.8720, 196557.1210, -254909.8720, 176692.9800, -136245.3974, 196557.1210, -136245.3974, 105056.8935])
m2 = m.LLL_gram()
print(m2)
P.<x,y,z> = ProjectiveSpace(QQ, 2)
X = P.subscheme(390908548757*x^4 - 1083699236751*x^3*y + 835578482044*x^3*z + 1126610184312*x^2*y^2 -
    1737329379412*x^2*y*z + 669777678687*x^2*z^2 - 520542386163*x*y^3 +
    1204081445939*x*y^2*z - 928398396271*x*y*z^2 + 238611653627*x*z^3 + 90192376558*y^4 -
    278168756247*y^3*z + 321720059816*y^2*z^2 - 165373310794*y*z^3 + 31877479532*z^4)
CR = P.coordinate_ring()
m2 = m2.change_ring(CR)
sub = m2*vector(CR.gens())
subs = {x: sub[0], y: sub[1], z: sub[2]}
X.defining_polynomials()[0].subs(subs)

[ -257    23    -7]
[-1303   118   -34]
[-1209   110   -31]


8406*x^4 - 4603*x^3*y + 948*x^2*y^2 - 87*x*y^3 + 3*y^4 + 232*x^3*z - 52*x^2*y*z + 3*x*y^2*z + 137*x^2*z^2 - 23*x*y*z^2 + y^2*z^2 + 36*x*z^3 - 3*y*z^3 + 3*z^4

In [6]:
m = matrix(ZZ, 3, 3, [367751, -254909, 196557, -254909, 176692, -136245, 196557, -136245, 105056])
m2 = m.LLL_gram()
print(m2)
P.<x,y,z> = ProjectiveSpace(QQ, 2)
X = P.subscheme(390908548757*x^4 - 1083699236751*x^3*y + 835578482044*x^3*z + 1126610184312*x^2*y^2 -
    1737329379412*x^2*y*z + 669777678687*x^2*z^2 - 520542386163*x*y^3 +
    1204081445939*x*y^2*z - 928398396271*x*y*z^2 + 238611653627*x*z^3 + 90192376558*y^4 -
    278168756247*y^3*z + 321720059816*y^2*z^2 - 165373310794*y*z^3 + 31877479532*z^4)
CR = P.coordinate_ring()
m2 = m2.change_ring(CR)
sub = m2*vector(CR.gens())
subs = {x: sub[0], y: sub[1], z: sub[2]}
X.defining_polynomials()[0].subs(subs)

[-2 -4 -1]
[11 -5 -3]
[18  1 -2]


1962*x^4 + 5377*x^3*y + 6123*x^2*y^2 + 3141*x*y^3 + 607*y^4 + 41*x^3*z + 4241*x^2*y*z + 3664*x*y^2*z + 979*y^3*z + 7235*x^2*z^2 + 2677*x*y*z^2 + 756*y^2*z^2 - 8354*x*z^3 - 372*y*z^3 + 4633*z^4

In [62]:
R = RealField(prec=200)
m = matrix(R, [[367751.9942459585261625773633578982341494732261957212701056845682640795311387705182237052975909844645, -254909.8719555023150149675612019645197795191670992469670823911868904298962919926095944074192352550851, 196557.1209821453833081250311216626803142243734635708009064766738165648444989409947114821253119785101],
    [-254909.8719555023150149675612019645197795191670992469670823911868904298962919926095944074192352550851, 176692.9799549980149890298705908710635901626574655981618344705560374970422384644235208105326228721947, -136245.3973548218806989026198330592048576998975562657267713522995456072062102321934555833597608727424],
    [196557.1209821453833081250311216626803142243734635708009064766738165648444989409947114821253119785101, -136245.3973548218806989026198330592048576998975562657267713522995456072062102321934555833597608727424, 105056.8934996086650519505817555127536378017083091688777628707679201997899901174738740965626108977313]])
LL = m.LLL_gram().transpose()
for i in LL:
    print(i)

(-7, -34, -31)
(23, 118, 110)
(89, 443, 408)


In [10]:
m = matrix(3, 3, [X.defining_polynomials()[0].derivative(CR.gens()[i]).derivative(CR.gens()[j]) for j in range(3) for i in range(3)])
Y = P.subscheme(m.det())
Z = X.intersection(Y)
C = ComplexField(prec=3000)
pnts = Z.rational_points(F=C)
ReduceCluster(pnts, precision=2000)

|der| = 681.7391686993723703875093051530378643582775815749394356053090537166556411258102201184484145978538253227559833595106105534456887227811815436701781257136034206072284986505523361996872320650863777331092095389794393001951342346107968210811915603803903376479750436693299079551546878058269392051594630611388713709185175681572573591701766223667433851415885866396639035451189039011587538572241629148244998260354565871691744855094042582270570540667029249535337937525767810123014066272359914809377192696470839569784330034996535176024393299441675435356468984970146576670749982764491273931431520445531215116756180


KeyboardInterrupt: 

In [15]:
Q = P.change_ring(C)
Q((0.2156691029155524194140809523120763063218114512491253540983303696122955787432321374298641566746515260,
       1.082226720555868698186377469661363975857759817369802676453885325359872841791295155441765675051518898,
       1.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000))

False

In [ ]:
ReduceCluster(pnts)

|der| = 681.59126499737750544584620422941562725802327768003486489371838124775674397007338936774768882407044388233034934233662115688580047633284561989577636338242046109654027414448049312363052017666290392471837008423382930748594937525258465838960105041613891719540168339349890237848875777294055280441349936448573257321803677264942332534722221246910062755518143687305676245540455149671943296375904250828181039165344406944995408933448380555124591866965108830058861770829211063523341595119668556768585423503422745788432879069940003609580431279424951178936496637275202624112472793362589648083403075261337197957882564622217953048450272798142836694141688749241420541394683990175624963886491340784826991832911689126345705768161175328242661975227427066914908387432921001


In [12]:
R.<u,v> = PolynomialRing(QQ, order='lex')
subs = {x: u, y:v, z:1}
ideal = R.ideal([poly.subs(subs) for poly in X.defining_polynomials()] + [poly.subs(subs) for poly in Y.defining_polynomials()])
polys = ideal.groebner_basis()

In [13]:
C = ComplexField(prec=2000)
roots2 = [tup[0] for tup in polys[1].univariate_polynomial().change_ring(C).roots()]
roots1 = [-1*polys[0](0, root) for root in roots2]


In [14]:
Q = P.change_ring(C)
pts = [Q((roots1[i], roots2[i], 1)) for i in range(len(roots1))]
pts

[(0.2156691029155524194140809523120763063218114706554413911905175767307051011740514342889362200869591602853886007330950389559649839409217402871811523680749412434251254475756794340752419380056015984254961936887034195863979268309974297540616478507838734786723726369819956762630124909950112449287781049534595730996365305572898613261592068375726879895623883339135919222407504600101717408974366428072439943841326536098092936555364791748350526434404799741002623379366606092658578964452730409428500771077867893379438054798167267921992718919219595450891665579753444427667221043651232076989976063552696919053037310 : 1.082226720555868698186377469661363975857759817369802676453885325359872841791295155441765675051519365651575969636302524861532813593803471704770001439590421861280985999791022743287643834544183355310328135574230782816750447711712466276832619539891017005040976452470785046004312728423231780907441977456885519020106943969991577424598469397652548167836992580270478233124460004372905896989761391194

In [24]:
cluster0 = [vector(list(pnt)) for pnt in pts]
cluster = [v*1/R(v.norm()) for v in cluster0]
cols = matrix(cluster).transpose()
cols[0]

(0.1448217556344408918765994291381461302582212905573288965509309934858793062380569462958884394951750851043740967939591790994612478435260861655565753974681993639186667116228653944913663480040054053368325090579060365611872356212832158523558666452866637231727088708815171656394936483759449342581185016871459006534069484238797599199008895597374182385459796927407596964428963908114597640986598117269479032744160148874400099947417064089063185816060384406599802756181811720176692329286460748713379902191057838211676907858504388171983945744130965304008075342367725570760361203488585273582814710198529019898497379, 0.14534837316516010421039538531750645795123527671938577353610179070503769253620858789191437666541680444526007916375219531509606976476796001804787035040461664660018757544074060638120853588468772218931384194989735516439981200674725388077806282364348441359380122115166534777830520601147102352078342930979412706738830192488954838906454913910346095913518552129339067377134080661881967567405151503113

In [15]:
ReduceCluster(pts, precision=2000)

|der| = 681.7391686993723703875093051530378643582775815749394356053090537166556411258102201184484145978538253227559833595106105534456887227811815436701781257136034206072284986505523361996872320650863777331092095389794393001951342346107968210811915603803903376479750436693299079551546878058269392051594630611388713709185175681572573591701766223667433851415885866396639035451189039011587538572241629148244998260354565871691744855094042582270570540667029249535337937525767810123014066272359914809377192696470839569784330034996535176024393299441675435356468974457857030753101562963211928426594304004801370751167796


KeyboardInterrupt: 

In [16]:
P.<x,y,z> = ProjectiveSpace(CC, 2)
pnts = [P((1,0,0)), P((1,1,1)), P((0,0,1)), P((0,1,0))]
ReduceCluster(pnts)

|der| = 1.33333333333333


KeyboardInterrupt: 

In [30]:
polys

[u + 25063221667140784101262928288185033304565552131595515022701886637807521198290688942341624906639095946751238015831293456674166249912683003664756536272685982332567012359685211163168166644283960020301016306379152922339727036754234177086452509348315371470844657726541905850371048394961319394589186529812073602247993368274355259605980927486164007396855019780355681854463265138431565287105585865284281417470154874223602974007651830242834665106239731606149280889526996968433580570465087442067251263970104333756414149085602181759133229369497773624987446420439334844785733406824092112505635435958891291613372533593796515851117244172245692694133433076500430401772473532182930586907887196876536664048723914259115128612391795508068473760683812391295017936206586061988298542110093922792769297914449384649809059709137545998594307231738029494136985639881072567238004757062918177610509964674622157625333647677131528468716274773909403599786500328274301853612329610933741554026517646466111091655079916223539725303

In [1]:
k = RR(1)
k.sign()

1

In [ ]:
t = RR(1)